### Скрипты для загрузки различных данных

Ниже приведены скриты для загрузки данных различных модальностей:
 - ADNI
 - UCLA
 - UCLA APOE
 - Parkinson
 
Каждый скрипт подгружает данные в виде матрицы размера n x k x k в переменную data и метки классов в переменную target размера 1 x n, где n - число объектов, k - количество вершин графа. В некоторых случаях
так же подгружается переменная patients_ids хранящия метки пациентов (размера 1 x n). Для загрузки данных необходимо указать путь к папке в переменной path (или path_to_read). Функция convert используется для конвертации матриц размера n x n в вектора размера (n x n - n) / 2 (без диагонали) или (n x n - n) / 2 + n (с диагональю) хранящие верхний треуголник матрицы, и обратно из вектора в симметричную матрицу. 

Скрипты написаны для Python 3.


In [1]:
import numpy as np
import pandas as pd
import os
from scipy.sparse import csr_matrix
from convert import convert

# ADNI

Данный набор данных содержит 807 снимков для 255 пациентов,
каждому снимку поставлен в соответствие граф размера с 68 вершинами,
метка класса (EMCI, Normal, AD, LMCI, SMC), а так же метка
пациентов (так как для каждого пациента есть несколько снимков,
метки класса для одного пациента одинаковы для всех его снимков)

In [2]:
path = 'connectomics/ADNI/Data/matrices/'

all_matrices = pd.DataFrame(columns = ['subject_id_file','subject_id','scan_id', 'matrix', 'target'])

# import data
for foldername in sorted(os.listdir(path)):
    for filename in sorted(os.listdir(path+foldername)):
        if 'NORM' not in filename:
            mat = np.genfromtxt(path+foldername+'/'+filename)
            subject_id_file = foldername
            subject_id = foldername[:-2]
            scan_id = foldername[-1:]
            
            # ADNI data have zeros on 3 and 38 row and column
            mat = np.delete(mat, [3,38], 1) 
            mat = np.delete(mat, [3,38], 0)
                        
            subject_data = convert(mat, mode = 'mat2vec')
            single_subject = pd.DataFrame(data = [[subject_id_file, subject_id, scan_id, subject_data, np.nan]],
                                          columns = ['subject_id_file','subject_id','scan_id', 'matrix', 'target'])
            all_matrices = all_matrices.append(single_subject)
            
all_matrices.index = all_matrices.subject_id_file
subject_data = pd.read_csv('connectomics/ADNI/Data/ADNI_subject_list.csv')
subject_id_names = np.array(all_matrices['subject_id_file'])

#importing target variables
for name in subject_id_names:
    smth = subject_data.loc[subject_data['Subject ID'] == name[:-2]]['DX Group'].dropna()
    un_smth = np.unique(smth)
    try:
        val = un_smth[0].replace(' ', '')
        all_matrices.set_value(name, 'target', val)
    except:
        pass
    
#drop objects without any target
all_matrices.dropna(inplace = True)
data_copy = all_matrices.copy()



temp = data_copy['matrix']

data_vectors = np.zeros((807, 2346))
data = np.zeros((807, 68, 68))

for idx, vec in enumerate(temp):
    data_vectors[idx] = vec
    data[idx] = convert(vec)
    
target = all_matrices.target.values
patients_ids = data_copy.subject_id.values

print('ADNI data shape                   :', data.shape,
     '\nADNI target variable shape        :', target.shape,
     '\nADNI number of unique patients    :', data_copy.subject_id.unique().shape)

ADNI data shape                   : (807, 68, 68) 
ADNI target variable shape        : (807,) 
ADNI number of unique patients    : (255,)


# UCLA

Набор данных UCLA содержит 94 снимка (людей с аутизмом и без), по 1 снимку для каждого пациента, каждому снимку поставлен в соответствие граф с 264 вершинами, переменная target содержит метки классов (1 - ASD - Аутизм, 0 - TD - норма)

In [3]:
path_to_read = 'connectomics/Autism/Data/dti/' #put your correct path here
target_vector = [] #this will be a target vector (diagnosis)
matrices=[] # this will be a list of connectomes 

for filename in sorted(os.listdir(path_to_read)): #for each file in a sorted (!) list of files:
    if "DTI_connectivity" in filename: #we only need files with DTI connectivity matrices
        if "All" not in filename: #we also do not need an average connectivity matrix here
            A_dataframe = pd.read_csv(path_to_read + filename, sep = '   ', header = None, engine = 'python')
            A = A_dataframe.values # we will use a list of numpy arrays, NOT pandas dataframes
            matrices.append(A) #append a matrix to our list
            if "ASD" in filename:
                target_vector.append(1)
            elif "TD" in filename: 
                target_vector.append(0)
data = np.array(matrices)
target = np.array(target_vector)

print('UCLA Autism data shape                   :', data.shape,
     '\nUCLA Autism target variable shape        :', target.shape)

UCLA Autism data shape                   : (94, 264, 264) 
UCLA Autism target variable shape        : (94,)


# UCLA APOE

Набор данных UCLA APOE содержит 55 снимков (людей носителей алелли а4, наличие которой повышает вероятность возникновения болезни Паркинсона, и без нее) по 1 снимку для каждого человека, каждому снимку поставлен в соответствие граф со 110 вершинами, переменная target содержит метки классов (1 - носитель, 0 - нет)


In [4]:
path_to_read = "UCLA_APOE_matrices/" #put your correct path here
target_vector_a4 = [] #this will be a target vector (diagnosis)
matrices_a4=[] # this will be a list of connectomes 

for filename in sorted(os.listdir(path_to_read)): #for each file in a sorted (!) list of files:
    if "connectivity" in filename: #we only need files with DTI connectivity matrices
        if "All" not in filename: #we also do not need an average connectivity matrix here
            A_dataframe = pd.read_csv(path_to_read + '/' + filename, sep = '   ', header = None, engine = 'python')
            A = A_dataframe.values # we will use a list of numpy arrays, NOT pandas dataframes
            matrices_a4.append(A) #append a matrix to our list
            if "APOE-4" in filename:
                target_vector_a4.append(1)
            elif "APOE-3" in filename: 
                target_vector_a4.append(0)
                
data = np.array(matrices_a4)
target = np.array(target_vector_a4)

print('UCLA APOE data shape                   :', data.shape,
     '\nUCLA APOE target variable shape        :', target.shape)

UCLA APOE data shape                   : (55, 110, 110) 
UCLA APOE target variable shape        : (55,)


# PARKINSON

Данный набор данных содержит 553 снимка для 295 пациентов,
каждому снимку поставлен в соответствие граф размера с 68 вершинами,
метка класса (Prodromal, SWEDD, PD, Control, GenCohortUnaff, GenCohortPD), а так же метка
пациентов (так как для каждого пациента есть несколько снимков,
метки класса для одного пациента одинаковы для всех его снимков <-- не проверено)

In [5]:
path = 'connectomics/Parkinson/Data/'

all_matrices = pd.DataFrame(columns = ['subject_id_file','subject_id',
                                       #'subject_id','scan_id',
                                       'matrix','centers', 'target'])
data = []

for foldername in sorted(os.listdir(path)):
    for filename in sorted(os.listdir(path+foldername)):
        if 'FULL' in filename:
            mat = np.genfromtxt(path+foldername+'/'+filename)
            subject_id_file = foldername
            subject_id = subject_id_file[:8]
            mat = mat[:70][:,:70]
            mat = np.delete(mat, [3,38], 1)
            mat = np.delete(mat, [3,38], 0)
            data.append(mat)
            subject_data = convert(mat,size = 68, mode = 'mat2vec')
            
        elif 'connect_grav' in filename:
            centers = pd.read_csv(path+foldername+'/'+filename)
            centers.drop([3,38], inplace=True)
            subject_center = np.array(centers[['mm_cordX', 'mm_cordY', 'mm_cordZ']])
                  
            
    single_subject = pd.DataFrame(data = [[subject_id_file, subject_id, subject_data, subject_center, np.nan]],
                                  columns = ['subject_id_file','subject_id', 'matrix','centers', 'target'])
    all_matrices = all_matrices.append(single_subject)
all_matrices.index = all_matrices.subject_id_file


meta_data = pd.read_csv('connectomics/Parkinson/demo_info.txt',header=None)
meta_data.columns = ['subject_id_file', 'id', 'date','age','sex', 'target']
meta_data.index = meta_data.subject_id_file

all_matrices.target = meta_data.target

data = np.array(data)
target = all_matrices.target.values
patients_ids = all_matrices.subject_id.values

print('Parkinson  data shape                  :', data.shape,
     '\nParkinson target variable shape        :', target.shape,
     '\nParkinson number of unique patients    :', all_matrices.subject_id.unique().shape)

Parkinson  data shape                  : (553, 68, 68) 
Parkinson target variable shape        : (553,) 
Parkinson number of unique patients    : (295,)
